# 01 — PyArrow + Parquet: schema e particionamento

Objetivo: entender Parquet na prática e criar dados particionados para leitura eficiente.

Tempo: ~25–30 min

In [ ]:
from pathlib import Path

def find_repo_root(start: Path | None = None) -> Path:
    """Sobe diretórios até encontrar uma 'marca' do repositório (README.md + pasta data)."""
    cur = (start or Path.cwd()).resolve()
    for _ in range(10):
        if (cur / "README.md").exists() and (cur / "data").exists():
            return cur
        cur = cur.parent
    # fallback: assume cwd
    return Path.cwd().resolve()

ROOT = find_repo_root()
DATA_DIR = ROOT / "data"
SAMPLE_DIR = DATA_DIR / "sample"

print("ROOT:", ROOT)
print("SAMPLE_DIR:", SAMPLE_DIR)

## 1) Pandas → Parquet com schema controlado

Vamos escrever um Parquet e inspecionar o schema com PyArrow.

In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

df = pd.read_csv(SAMPLE_DIR / "sales.csv")

table = pa.Table.from_pandas(df)
table.schema

## 2) Escrevendo Parquet e lendo schema

In [ ]:
out_dir = ROOT / "data" / "output"
out_dir.mkdir(parents=True, exist_ok=True)

parquet_path = out_dir / "sales_arrow.parquet"
pq.write_table(table, parquet_path)

pq.read_schema(parquet_path)

## 3) Dataset particionado (ex.: por região)

Particionar ajuda leituras seletivas (ex.: só uma região).

In [ ]:
import pyarrow.dataset as ds

partition_dir = out_dir / "sales_partitioned"
partition_dir.mkdir(parents=True, exist_ok=True)

ds.write_dataset(
    data=table,
    base_dir=str(partition_dir),
    format="parquet",
    partitioning=["region"],
    existing_data_behavior="overwrite_or_ignore"
)

partition_dir

## 4) Lendo só uma partição

In [ ]:
dataset = ds.dataset(str(partition_dir), format="parquet")

# Filtra por região sem ler tudo
filtered = dataset.to_table(filter=ds.field("region") == "Sudeste")
filtered.num_rows, filtered.schema.names

## Exercícios

1- Particione por `category`.
2- Compare o tamanho do Parquet único vs dataset particionado.
3- Leia duas regiões e faça uma agregação em PyArrow (soma de revenue).

In [ ]:
# Escreva suas respostas aqui
